# **Install MMDetection**

In [ ]:
!nvcc -V
!gcc --version

In [ ]:
!pip install -U torch==1.7.1+cu110 torchvision==0.8.2+cu110 -f https://download.pytorch.org/whl/torch_stable.html
!pip install mmcv-full
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection
%cd mmdetection
!pip install -e .

# **Import Libraries**

In [ ]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

import mmdet
print(mmdet.__version__)

from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())
import os
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import glob
import cv2
import shutil
import random
import os.path as osp
import json
import mmcv
import random
import re
import xml.etree.ElementTree as ET
from typing import Dict, List
from mmdet.apis import set_random_seed
import pandas as pd
import numpy as np
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

# **Set Seeds**

In [ ]:
global_seed = 0

def set_seed(seed=global_seed):
    """Sets the random seeds."""
    set_random_seed(seed, deterministic=False)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed()

In [ ]:
%cd ..

# **Turn CSV to COCO Annotation Format**

In [ ]:
df = pd.read_csv('../input/license/license.csv')

In [ ]:
val_idx = random.sample(range(0, 236), 20)

In [ ]:
len(val_idx)

In [ ]:
idx = []
for i in range(237):
    idx.append(i)
train_idx = np.setdiff1d(idx, val_idx)

In [ ]:
train_idx = train_idx.tolist()

In [ ]:
len(train_idx)

In [ ]:
val_idx.remove(107) #For some reason cv2 can't read the image correctly so I just removed it. 

In [ ]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

In [ ]:
!mkdir /kaggle/working/images
!cp -r ../input/license/Cars/. /kaggle/working/images

In [ ]:
%%writefile labels.txt
License_Plate

In [ ]:
def convert_df_to_cocojson(df, idx_list, output_jsonpath='/kaggle/working/output.json', label='License_Plate'):
    output_json_dict = {
        "images": [],
        "annotations": [],
        "categories": []

    }
    for i, idx in enumerate(idx_list):
        img_filepath = df.iloc[idx, 1].split('/')[-1]
        xmin = df.iloc[idx, 2]
        ymin = df.iloc[idx, 3]
        xmax = df.iloc[idx, 4]
        ymax = df.iloc[idx, 5]
        ids = idx
#         print(img_filepath)
        img_height, img_width, img_depth = cv2.imread('/kaggle/working/images/' + img_filepath).shape
        real_xmin = int(xmin*img_width)
        real_xmax = int(xmax*img_width)
        real_ymin = int(ymin*img_height)
        real_ymax = int(ymax*img_height)   
        annot_width = int(real_xmax - real_xmin)
        annot_height = int(real_ymax - real_ymin)
        annot = {
            'segmentation': [], 
            'area': annot_width * annot_height,
            'iscrowd': 0,
            'image_id': int(idx + 1),
            'bbox': [real_xmin, real_ymin, annot_width, annot_height],
            'category_id': 1,
            'id': int(i + 1)
        }
        output_json_dict['annotations'].append(annot)
        image_info = {
            'file_name': img_filepath,
            'height': img_height,
            'width': img_width,
            'id': int(idx + 1)
        }
        output_json_dict['images'].append(image_info)
    category_info = {
        'id': 1, 
        'name': label, 
        'supercategory': 'none'
    }
    output_json_dict['categories'].append(category_info)
    with open(output_jsonpath, 'x') as f:
        output_json = json.dumps(output_json_dict, cls=NpEncoder)
        f.write(output_json)

In [ ]:
convert_df_to_cocojson(df=df, idx_list=train_idx)

In [ ]:
convert_df_to_cocojson(df=df, idx_list=val_idx, output_jsonpath='/kaggle/working/val-output.json')

# **Custom Config File with Deformable CNN V2**

In [ ]:
from mmcv import Config
cfg = Config.fromfile('/kaggle/working/mmdetection/configs/dcn/cascade_rcnn_r101_fpn_dconv_c3-c5_1x_coco.py')

In [ ]:
cfg.dataset_type = 'CocoDataset'
cfg.classes = '/kaggle/working/labels.txt'
cfg.data_root = '/kaggle/working'

for head in cfg.model.roi_head.bbox_head:
    head.num_classes = 1
    
cfg.work_dir = './model_output'

cfg.data.test.type = 'CocoDataset'
cfg.data.test.classes = 'labels.txt'
cfg.data.test.data_root = '/kaggle/working'
cfg.data.test.ann_file = 'val-output.json'
cfg.data.test.img_prefix = 'images'

cfg.data.train.type = 'CocoDataset'
cfg.data.train.data_root = '/kaggle/working'
cfg.data.train.ann_file = 'output.json'
cfg.data.train.img_prefix = 'images'
cfg.data.train.classes = 'labels.txt'

cfg.data.val.type = 'CocoDataset'
cfg.data.val.data_root = '/kaggle/working'
cfg.data.val.ann_file = 'val-output.json'
cfg.data.val.img_prefix = 'images'
cfg.data.val.classes = 'labels.txt'

albu_train_transforms = [
    dict(type='ShiftScaleRotate', shift_limit=0.0625,
         scale_limit=0.15, rotate_limit=15, p=0.4),
#     dict(type='RandomBrightnessContrast', brightness_limit=0.2,
#          contrast_limit=0.2, p=0.5),
    dict(type='IAAAffine', shear=(-10.0, 10.0), p=0.4),
#     dict(type='MixUp', p=0.2, lambd=0.5),
#     dict(type="Blur", p=1.0, blur_limit=7),
#     dict(type='CLAHE', p=0.5),
#     dict(type='Equalize', mode='cv', p=0.4),
    dict(
        type="OneOf",
        transforms=[
            dict(type="GaussianBlur", p=1.0, blur_limit=7),
            dict(type="MedianBlur", p=1.0, blur_limit=7),
        ],
        p=0.4,
    ),]

cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Albu',
        transforms=albu_train_transforms,
        bbox_params=dict(
        type='BboxParams',
        format='coco',
        label_fields=['gt_labels'],
        min_visibility=0.0,
        filter_lost_elements=True),
        keymap=dict(img='image', gt_bboxes='bboxes'),
        update_pad_shape=False,
        skip_img_without_anno=True),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]

cfg.load_from = '../input/dcn-checkpoint/cascade_rcnn_r101_fpn_dconv_c3-c5_1x_coco_20200203-3b2f0594.pth'

cfg.work_dir = '/kaggle/working/model_output'

cfg.optimizer.lr = 0.02 / 8
cfg.lr_config = dict(
    policy='CosineAnnealing', 
    by_epoch=False,
    warmup='linear', 
    warmup_iters=500, 
    warmup_ratio=0.001,
    min_lr=1e-07)

cfg.data.samples_per_gpu = 4
cfg.data.workers_per_gpu = 2

cfg.evaluation.metric = 'bbox'
cfg.evaluation.interval = 4

cfg.checkpoint_config.interval = 6
cfg.runner.max_epochs = 12
cfg.log_config.interval = 50

cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = [0]

print(f'Config:\n{cfg.pretty_text}')

# **Training**

In [ ]:
datasets = [build_dataset(cfg.data.train)]
model = build_detector(
 cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
model.CLASSES = datasets[0].CLASSES

mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

We can keep training for more accuracy, but I chose not to. 

# **Inference on Validation Set**

In [ ]:
model = init_detector(cfg, '/kaggle/working/model_output/epoch_12.pth')
for i in range(len(val_idx)):
    img = mmcv.imread('/kaggle/input/license/Cars/car' + str(val_idx[i]) + '.jpg')
    result = inference_detector(model, img)
    show_result_pyplot(model, img, result)